In [ ]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import *

In [ ]:
X_train = [[-1, -1],  [-1,1], [1,1], [1,-1]]
y_train = [1, 0, 1, 0]

X_test =  [[-1, -1],  [-1,1], [1,1], [1,-1]]
y_test =  [1, 0, 1, 0]

## Test

In [ ]:
w = list(np.random.uniform(low=0.0, high=1.0, size=(len(X_test[0]),)))

In [ ]:
weights = quantumNeuronFIT(X_train, 
                           y_train,
                           init_weight = w,
                           threshold=0.7,
                           lrParameter=0.1, 
                           n_epochs=1, 
                           trainingBias=False,
                           epoch_results=False,
                           phaseEstrategyOperator='original',
                           trainingApproaches={'hsgs'})

In [ ]:
results = quantumNeuronPREDICT(X_test, 
                                   y_test,
                                   threshold=0.7,
                                   weightVectorsPhaseEncoding = weights[0], 
                                   weightVectorsHSGS = weights[1], 
                                   repeat=1,
                                   bias=False, 
                                   phaseEstrategyOperator='original',
                                   testingApproaches={'hsgs'})

In [ ]:
results

## Binary Quantum Neuron Experiments 

In [ ]:
output = {'model':[],
          'phase_strategy':[],
          'threshold':[],
            'bias':[],
            'lr':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [ ]:
# search space
initial_weights =  list(np.random.uniform(low=0.0, high=1.0, size=(len(X_train[0]),)))
threshold_space = [0.3, 0.5, 0.7, 0.8, 0.9]
lr_space = [0.02, 0.1]
bias_space = [False, True]
strategy_space = ['original']

In [ ]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=initial_weights,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=100, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'hsgs'})

                    results = quantumNeuronPREDICT(X_test, 
                                                   y_test,
                                                   threshold=threshold,
                                                   weightVectorsPhaseEncoding = weights[0], 
                                                   weightVectorsHSGS = weights[1], 
                                                   repeat=5,
                                                   bias=bias, 
                                                   phaseEstrategyOperator=strategy,
                                                   testingApproaches={'hsgs'})


                    output['model'].append('HSGS')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['neuron_outputs'].append(results[1])
                    output['initial_weights'].append(w)
                    print('\n')

In [ ]:
experiment_hsgs = pd.DataFrame(output)

In [ ]:
experiment_hsgs.to_csv('data_and_results/XOR/XOR_probs_hsgs.csv', index=False)

## Continuously Valued Quantum Neuron Experiments

In [ ]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [ ]:
# search space
initial_weights =  list(np.random.uniform(low=0.0, high=1.0, size=(len(X_test[0]),)))
threshold_space = [0.3, 0.5, 0.7, 0.9]
lr_space = [0.02, 0.1]
bias_space = [False, True]
strategy_space = ['original', 'angle', 'radius', 'angleradius']

In [ ]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=initial_weights,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=100, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'phase-encoding'})

                    results = quantumNeuronPREDICT(X_test, 
                                                       y_test,
                                                       threshold=threshold,
                                                       weightVectorsPhaseEncoding = weights[0], 
                                                       weightVectorsHSGS = weights[1], 
                                                       repeat=5,
                                                       bias=bias, 
                                                       phaseEstrategyOperator=strategy,
                                                       testingApproaches={'phase-encoding'})

                    # get experiment results
                    output['model'].append('phase-encoding')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['neuron_outputs'].append(results[0])
                    output['initial_weights'].append(w)
                    print('\n')

In [ ]:
experiment_phase = pd.DataFrame(output)

### Save Results

In [ ]:
experiments = pd.concat([experiment_hsgs, experiment_phase])

In [ ]:
experiments.to_csv('data_and_results/XOR/experiments_XOR_probs.csv')

In [ ]:
xore = pd.read_csv('data_and_results/XOR/experiments_XOR_encoding_probs.csv')
xor = pd.read_csv('data_and_results/XOR/experiments_XOR_probs.csv')

In [ ]:
experiments = pd.concat([xor, xore])

In [ ]:
experiments.to_csv('data_and_results/XOR/experiments_XOR_probs.csv', index=False)